In [14]:
from connector import MT5
import pandas as pd
import numpy as np

from datetime import datetime
from decimal import *
import pytz

x = MT5.get_tickers()
timezone = pytz.utc

tickers = []

for t in x:
    tickers.append({"ticker": t.name, "digits": t.digits})

tickers = tickers[:1]

candles = MT5.get_candles(
    {"ticker": tickers[0]["ticker"], "timeframe": "5M", "count": 10_000}
)

print(tickers)

Candles Fetched:  07/15/2023, 05:45:30
[{'ticker': 'EURUSD', 'digits': 5}]


In [18]:
pd.set_option("display.max_rows", None)


def get_candles_df(candles):
    _candles = pd.DataFrame(
        candles,
        # columns=['Datetime', 'Open', 'High', 'Low', 'Close', 'tick_volume', 'spread', 'real_volume']
    )

    _candles["Datetime"] = _candles["time"].map(
        lambda x: datetime.fromtimestamp(x, timezone)
    )

    _candles = _candles.set_index("Datetime")

    return _candles


def predict_crossover(params):
    df = params["df"]

    df["small_ema"] = df["close"].ewm(span=params["small_ema"], adjust=False).mean()
    df["large_ema"] = df["close"].ewm(span=params["large_ema"], adjust=False).mean()

    df["small_ema"] = round(df["small_ema"], params["round_factor"])
    df["large_ema"] = round(df["large_ema"], params["round_factor"])

    df["cross_diff"] = df["large_ema"] - df["small_ema"]

    df["crossover_type"] = np.where(df["cross_diff"] > 0, "short", "long")

    df["diff"] = abs(df["cross_diff"])

    df["crosspoint"] = np.where(
        df["crossover_type"] != df["crossover_type"].shift(1),
        True,
        False,
    )

    return df[params["large_ema"] :]


candles = get_candles_df(candles)

df = predict_crossover(
    {
        "df": candles,
        "small_ema": 8,
        "large_ema": 38,
        "round_factor": tickers[0]["digits"],
    }
)

dfx = df[df["crosspoint"] == True]
dfs = dfx[dfx["crossover_type"] == "short"]
dfl = dfx[dfx["crossover_type"] == "long"]

stats = {
    "ticker": tickers[0]["ticker"],
    "digits": tickers[0]["digits"],
    "crossovers_count": len(dfx),
    "longovers": len(dfl),
    "shortovers": len(dfs),
}

thresher = []

for i in range(0, len(df)):
    if df.iloc[i]["crosspoint"] == True:
        thresher.append(df.iloc[i - 3])

freq = {}

for candle in thresher:
    diff = candle["diff"]
    round_factor = stats["digits"]

    key = int(round(diff, round_factor) * pow(10, round_factor))

    if key in freq:
        freq[key] = freq[key] + 1
    else:
        freq[key] = 1

print(freq)

max = [0, 0]

for key in freq:
    if freq[key] > max[1]:
        max = [key, freq[key]]

print("Max", max)

print(thresher)

{17: 7, 12: 31, 4: 27, 5: 19, 2: 37, 1: 14, 3: 20, 20: 7, 27: 6, 49: 3, 10: 19, 9: 9, 76: 1, 71: 1, 16: 10, 19: 6, 29: 6, 55: 1, 23: 5, 11: 13, 6: 26, 0: 7, 21: 7, 31: 2, 18: 5, 13: 9, 42: 2, 8: 11, 24: 4, 36: 2, 32: 3, 22: 6, 89: 1, 14: 12, 33: 1, 47: 1, 25: 7, 85: 1, 62: 1, 40: 1, 39: 2}
Max [2, 37]
[time              1685354400
open                 1.07362
high                 1.07363
low                  1.07308
close                1.07337
tick_volume              487
spread                     8
real_volume                0
small_ema             1.0738
large_ema            1.07363
cross_diff          -0.00017
crossover_type          long
diff                 0.00017
crosspoint             False
Name: 2023-05-29 10:00:00+00:00, dtype: object, time              1685385000
open                 1.07079
high                 1.07102
low                  1.07079
close                1.07102
tick_volume              121
spread                     8
real_volume                0
small_ema 